In [ ]:
import numpy as np
from numba import jit
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
import tensorflow as tf
from tensorflow.python.ops.numpy_ops import np_config
import tensorflow_datasets as tfds
np_config.enable_numpy_behavior()

drive.mount('/content/drive')

def XYZtoUVWT(data):
    referencePoint = np.array([-138.9971, 98.25])
    phi = np.pi/6.0
    stripPitch = 1.5
    f = 1.0/25*6.46
    u = -(data[1]-99.75)
    v = (data[0]-referencePoint[0])*np.cos(phi) - (data[1]-referencePoint[1])*np.sin(phi)
    w = (data[0]-referencePoint[0])*np.cos(-phi) - (data[1]-referencePoint[1])*np.sin(-phi) + 98.75
    t = data[2]/f + 256
    u/=stripPitch
    v/=stripPitch
    w/=stripPitch
    return np.array((u,v,w,t)).T

def dataset_generator(dataset_tf):
  n_projections = 3
  scale = 100
  for record in dataset_tf:
    images, targets = record
    uvwt_1 = XYZtoUVWT(scale*targets[0:3])
    uvwt_2 = XYZtoUVWT(scale*targets[3:6])
    uvwt_3 = XYZtoUVWT(scale*targets[6:9])
    for projection in range(n_projections):
      points = np.array([
          uvwt_1[3],
          uvwt_1[projection],
          uvwt_2[3],
          uvwt_2[projection],
          uvwt_3[3],
          uvwt_3[projection]
          ])
      yield images[:, :, projection], points

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
dataPath = 'drive/MyDrive/ZPS/simulated_data/'
train_files = [dataPath+'out_random_sigma-001.root:TPCData']
train_dataset = tf.data.TFRecordDataset(dataPath + 'out_random_sigma-001.tfrecord', compression_type='GZIP')
test_dataset = tf.data.TFRecordDataset(dataPath + 'out_random_sigma2k2mm.tfrecord', compression_type='GZIP')
feature_description = {
    'myChargeArray': tf.io.FixedLenFeature([], tf.string),
    'target': tf.io.FixedLenFeature([], tf.string),

}
def _parse_function(example_proto):
    parsed_features = tf.io.parse_single_example(example_proto, feature_description)
    charge, target = parsed_features['myChargeArray'], parsed_features['target']
    # decode from bytes
    charge = tf.io.decode_raw(charge, 'float64')
    target = tf.io.decode_raw(target, 'float64')
    # reshape
    charge = charge.reshape([256, 512, 3])
    target = target.reshape([9])

    return charge, target


train_dataset_tf = tfds.as_numpy(train_dataset.map(_parse_function, num_parallel_calls=tf.data.AUTOTUNE))
test_dataset_tf = tfds.as_numpy(test_dataset.map(_parse_function, num_parallel_calls=tf.data.AUTOTUNE))

In [ ]:
gen = dataset_generator(test_dataset_tf)

In [ ]:
test_folder = "drive/MyDrive/Data/Test"
train_folder = "drive/MyDrive/Data/Train"

In [ ]:
import time
from tqdm import tqdm
start_time = time.time()  # Record the initial start time
save_folder = ''
for index, data in enumerate(gen):
  img, target = data
  np.save(f"{save_folder}/img_{index}.npy", img)
  np.save(f"{save_folder}/t_{index}.npy", target)

6000it [00:51, 117.51it/s]


In [ ]:
#for i, test_data in enumerate(data_list[:300]):
#  img, target = test_data
#  np.save(f"{test_folder}/img_{i}.npy", img)
#  np.save(f"{test_folder}/t_{i}.npy", target)
#
#for i, train_data in enumerate(data_list[300:]):
#  img, target = train_data
#  np.save(f"{train_folder}/img_{i}.npy", img)
#  np.save(f"{train_folder}/t_{i}.npy", target)